In [1]:
!pip install streamlit pyngrok google-cloud-aiplatform -q

In [8]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import os

# --- Configuration ---
st.set_page_config(layout="wide", page_title="Veritas AI")

# Get API key from the environment variable set in the launch cell
GCP_API_KEY = userdata.get('GCP_API_KEY')

# Configure the Generative AI client
if API_KEY:
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-2.5-pro')
else:
    model = None

# --- Function to call Gemini AI ---
def get_veritas_analysis(content):
    """
    Sends content to the Gemini Pro model for misinformation analysis.
    """
    # This is the "prompt" that instructs the AI
    prompt = f"""
    Analyze the following text for misinformation. Provide your analysis in three parts:
    1.  **Credibility Score:** A score from 0 (completely false) to 100 (highly credible).
    2.  **Summary:** A brief, neutral summary of the main claims in the text.
    3.  **Red Flags:** A bulleted list of any manipulative language, logical fallacies, lack of sources, or other red flags you detect. If none, say "No significant red flags detected."

    Here is the text to analyze:
    ---
    {content}
    ---
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        # Provide a more user-friendly error message
        error_message = str(e)
        if "API key not valid" in error_message:
            return "An error occurred: Your Google Cloud API key is not valid. Please check it in the Colab Secrets."
        else:
            return f"An error occurred: {error_message}"

# --- Streamlit User Interface ---
st.title("🛡️ Veritas AI – Your Shield Against Deception")
st.write("An AI-powered tool to analyze text for potential misinformation and manipulative language.")

user_input = st.text_area("Paste the text or article you want to analyze:", height=250)

if st.button("🔎 Analyze Content"):
    if not user_input:
        st.error("Please paste some text to analyze.")
    elif not model:
        st.error("Google Cloud API Key not configured. Please set the 'GCP_API_KEY' in your Colab Secrets and restart the runtime.")
    else:
        with st.spinner('AI is analyzing the content... This may take a moment.'):
            analysis_result = get_veritas_analysis(user_input)

            st.subheader("Analysis Report")
            st.markdown(analysis_result)

Overwriting app.py


In [9]:
import os
from pyngrok import ngrok
from google.colab import userdata

# --- Get Ngrok token from Colab secrets ---
NGROK_TOKEN = userdata.get('NGROK_AUTHTOKEN')
#GCP_API_KEY = userdata.get('GCP_API_KEY')
# --- Kill any old processes to ensure a clean start ---
!pkill -f streamlit
!pkill -f ngrok

# --- Start Streamlit in the background ---
# nohup keeps it running, and &> saves output to a log file for debugging
!nohup streamlit run app.py --server.port 8501 &> streamlit.log &

# --- Launch the Ngrok tunnel to make the app public ---
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(addr="8501", proto="http")

print("================================================================================")
print("✅ Your Veritas AI prototype is LIVE!")
print(f"🌍 Public URL: {public_url}")
print("================================================================================")
print("NOTE: This cell must remain running to keep your app live.")

# This loop keeps the Colab cell running, which keeps your app alive.
try:
    while True:
        pass
except KeyboardInterrupt:
    print("\nShutting down...")
    ngrok.kill()

✅ Your Veritas AI prototype is LIVE!
🌍 Public URL: NgrokTunnel: "https://923bfb29c10a.ngrok-free.app" -> "http://localhost:8501"
NOTE: This cell must remain running to keep your app live.

Shutting down...
